# CH3 - 텍스트와 바이트
- 문자, 코드 포인트, 바이트 표현
- 이진 시퀀스의 고유한 특징 : bytes, bytearray, memoryview
- 전체 유니코드 및 레거시 문자셋에 대한 코덱
- 인코딩 에러를 피하고 다루기
- 텍스트 파일을 다룰 때 모범 사례
- 기본 인코딩 및 표준 입출력 문제
- 정규화를 이용한 안전한 유니코드 텍스트 비교
- 정규화, 케이스 폴딩, 발음 구별 기호 강제 제거를 위한 유틸리티 함수
- locale과 PyUCA 라이브러리를 이용한 유니코드 텍스트의 적절한 정렬
- 유니코드 데이터베이스 안의 문자 메타데이터
- str과 bytes 를 다루는 이중 모드 API

## 4.1 문자 문제
- 파이썬3 에서는 str에서 가져오는 항목이 유니코드 문자이다.
- 유니코드 표준은 문자의 단위 원소와 특정 바이트 표현을 명확히 구분한다.

### 단위 원소(코드 포인트)
- 문자에 부여한 고유한 숫자
- 10진수 0에서 1,114,111까지의 숫자

### 인코딩, 디코딩
- 코드포인드 -> 바이트 : 인코딩, 바이트 -> 코드 포인트 : 디코딩
- 실제 바이트는 사용하는 인코딩에 따라 달라진다
    - A(U+0041) 에 대한 UTF-8 : 1바이트 \x41
    - A(U+0041) 에 대한 UTF-16LE : 2바이트 \x41\x00

In [4]:
s = 'café'
print(len(s))

b = s.encode('utf8')
print(b)
print(len(b))

b = b.decode('utf8')
print(b)
print(len(b))

4
b'caf\xc3\xa9'
5
café
4


## 4.2 바이트
- 이진 시퀀스를 위한 자료형 : bytes(불변), bytearray(가변)
- 각 항목은 0 ~ 255 사이의 정수(1바이트)

In [34]:
cafe = bytes('café', encoding='utf_8')
print(cafe)
print(cafe[4])

# 슬라이싱이 가능하다. 슬라이싱한 결과물도 bytes, bytearray형이다
print(cafe[:1])

cafe_arr = bytearray(cafe)
print(cafe_arr)
print(cafe_arr[-1:])

# 16진수 쌍으로 이진 시퀀스 생성
print(bytes.fromhex('31 4B CE A9'))

b'caf\xc3\xa9'
169
b'c'
bytearray(b'caf\xc3\xa9')
bytearray(b'\xa9')
b'1K\xce\xa9'


In [32]:
a = bytes('print\n', encoding='utf_8')

In [33]:
a

b'print\n'

In [3]:
import array
# 'h' -> short int 형(2 bytes) 시퀀스 생성
numbers = array.array('h', [-2, -1, 0, 1, 2])
print(numbers)
octets = bytes(numbers)
print(octets)

array('h', [-2, -1, 0, 1, 2])
b'\xfe\xff\xff\xff\x00\x00\x01\x00\x02\x00'


### 4.2.1 구조체와 메모리 뷰
#### struct 모듈
- 패킹된 바이트를 다양한 형의 필드로 구성된 튜플로 분석
- 튜플을 패킹된 바이트로 변환

In [16]:
import struct
fmt = '<3s3sHH'
with open ('nyan-cat.gif', 'rb') as fp:
    img = memoryview(fp.read())

header = img[:10]
print(bytes(header))
# 종류, 버전, 너비, 높이 -> 3바이트, 3바이트, 16비트 정수, 16비트 정수 로 이루어진다! -> 3s3sHH 
print(struct.unpack(fmt, header))
del header
del img

b'GIF89a\xdc\x01\x18\x01'
(b'GIF', b'89a', 476, 280)


### 4.4 인코딩/디코딩 문제 이해
- UnicodeError 범용 에러 존재
- 하지만 거의 항상 UnicodeEncodeError, UnicodeDecodeError 같은 구체적인 예외가 발생한다

#### 4.4.1 UnicodeEncodeError 처리하기
- 대부분의 비 UTF 코덱은 유니코드 문자의 일부분만 처리할 수 있다.
- 텍스트를 바이트로 변환 시, 문자가 대상 인코딩에 정의되어 있지 많으면 별도의 처리를(errors 인수) 하지 않는 한 에러가 발생한다

In [25]:
city = 'São Pailo?'
print('utf-8', city.encode('utf-8'), sep='\t')
print('utf-16', city.encode('utf-16'), sep='\t')
# print('cp437', city.encode('cp437'), sep='\t')
# ignore : 처리할 수 없는 문자를 건너뜀, 안 좋은 방법
print('cp437', city.encode('cp437', errors='ignore'), sep='\t')
# replace : 처리할 수 없는 문자를 ? 로 치환
print('cp437', city.encode('cp437', errors='replace'), sep='\t')
# 인코딩할 수 없는 문자를 XML 개체로 치환
print('cp437', city.encode('cp437', errors='xmlcharrefreplace'), sep='\t')



utf-8	b'S\xc3\xa3o Pailo?'
utf-16	b'\xff\xfeS\x00\xe3\x00o\x00 \x00P\x00a\x00i\x00l\x00o\x00?\x00'
cp437	b'So Pailo?'
cp437	b'S?o Pailo?'
cp437	b'S&#227;o Pailo?'


#### 4.4.2 UnicodeDecodeError 처리하기
- 모든 바이트 시퀀스가 정당한 UTF 형식 문자가 되는 것은 아니다
- 변환 불가능할 때 UnicodeDecodeError 가 발생한다
    - cp1252, koi8_r 등 레거시 8비트 코덱은 무작위 비트 배열에 대해서도 에러를 발생시키지 않고 바이트 스트림으로 디코딩 할 수 있다
    - 즉, 잘못된 8비트 코덱을 사용하면 쓰레기 문자를 디코딩하게 된다.

In [38]:
# \xe9 : é
octets = b'Montr\xe9al'
# Windows OS 에서 많이 사용한다
print(octets.decode('cp1252'))
print(octets.decode('iso8859_7'))
print(octets.decode('koi8_r'))
# print(octets.decode('utf-8'))
print(octets.decode('utf_8', errors='replace'))
print('Montréal'.encode('utf-8'))

Montréal
Montrιal
MontrИal
Montr�al
b'Montr\xc3\xa9al'


#### 4.4.3 인코딩 된 모듈을 로딩할 떄 발생하는 SyntaxError
- 파이썬3 은 기본적으로 UTF-8을 소스 코드 기본 인코딩으로 사용한다
- 비 UTF-8 로 인코딩된 .py 모듈 로딩 시 SyntaxError가 발생하게 된다
    - 파일 맨 위에 `# coding : <codec>` 주석을 달아 해결 가능하다

#### 4.4.4 바이트 시퀀스의 인코딩 방식을 알아내기
- 불가능하다. 반드시 별도로 인코딩 정보를 가져와야 한다
- 추정은 가능하다. Chardet 패키지는 30가지 인코딩 방식을 알아낼 수 있다

### 4.5 텍스트 파일 다루기
- 유니코드 샌드위치
    - 입력할 때 (파일을 여는 때 등) 가능하면 빨리 bytes를 str로 변환해야 한다는 것을 의미
    - 즉, bytes -> str / str 로 파일을 다룸(처리 중 인코딩이나 디코딩하면 안 됨) / str -> bytes(내보냄)

In [39]:
# 플랫폼 인코딩 문제
open('cafe.txt', 'w', encoding='utf_8').write('café')
print(open('cafe.txt').read())

"""
Ubuntu 는 기본 인코딩이 utf-8이기 때문에 아무런 문제가 없다
하지만 윈도우에서 실행했을 때는 문제가 생길 것이다. (시스템 기본 인코딩이 Windows 1252 이기 때문)
파일을 쓸 때 encoding 인수를 생략하면 기본 지역 설정에 따른 인코딩 방식을 사용하고, 읽을 때도 동일한 인코딩 방식으로 올바르게 읽을 수 있다

여러 컴퓨터나 여러 상황에서 실행되어야 하면 기본 인코딩에 의존하면 안 된다. 언제나 encoding 인수를 명시적으로 지정해야 한다.
"""

café


In [56]:
fp = open('cafe.txt', 'w', encoding='utf_8')
print(fp)
# write는 저장한 "유니코드" 문자 수를 반환한다
print(fp.write('café'))
fp.close()

import os
print(os.stat('cafe.txt').st_size, '\n')

fp2 = open('cafe.txt')
print(fp2)
print(fp2.encoding)
print(fp2.read(), '\n')

fp3 = open('cafe.txt', encoding='cp1252')
print(fp3)
print(fp3.read(), '\n')

fp4 = open('cafe.txt', 'rb')
print(fp4)
print(fp4.read(), '\n')

fp2.close()
fp3.close()
fp4.close()

<_io.TextIOWrapper name='cafe.txt' mode='w' encoding='utf_8'>
4
5 

<_io.TextIOWrapper name='cafe.txt' mode='r' encoding='UTF-8'>
UTF-8
café 

<_io.TextIOWrapper name='cafe.txt' mode='r' encoding='cp1252'>
cafÃ© 

<_io.BufferedReader name='cafe.txt'>
b'caf\xc3\xa9' 



**tips**
- 인코딩 방식을 알아내기 위해 내용을 분석하는 게 아니라면 텍스트 파일을 이진 모드로 열지 않는 게 좋다
- 인코딩 방식을 알아낼 때도 직접 하는 것 보다 모듈을 이용해라 (Chardet)
- 래스터 이미지 등 이진 파일을 열 때만 이진 모드를 사용해라

In [59]:
import sys, locale

expressions = """
    locale.getpreferredencoding()
    type(my_file)
    sys.stdout.isatty()
    sys.stdout.encoding
    sys.stdin.isatty()
    sys.stdin.encoding
    sys.stderr.isatty()
    sys.stderr.encoding
    sys.getdefaultencoding()
    sys.getfilesystemencoding()
    """

my_file = open('dummy', 'w')

for exp in expressions.split():
    value = eval(exp)
    print(exp.rjust(30), '->', repr(value))

 locale.getpreferredencoding() -> 'UTF-8'
                 type(my_file) -> <class '_io.TextIOWrapper'>
           sys.stdout.isatty() -> False
           sys.stdout.encoding -> 'UTF-8'
            sys.stdin.isatty() -> False
            sys.stdin.encoding -> 'UTF-8'
           sys.stderr.isatty() -> False
           sys.stderr.encoding -> 'UTF-8'
      sys.getdefaultencoding() -> 'utf-8'
   sys.getfilesystemencoding() -> 'utf-8'


- 기본 인코딩에 의존하지 않는 게 가장 좋다
- 리눅스, 맥의 경우 UTF-8 이 가본적이로 사용되어 오류가 적을 수 있지만 윈도우의 경우에는 여러 인코딩들이 사용된다.
- 따라서 유니코드 샌드위치 모델을 따르고, 인코딩을 늘 명시하면 많은 문제를 피할 수 있다.

### 4.6 제대로 비교하기 위해 유니코드 정규화하기

